In [121]:
#IMPORT
import random
import hashlib
from itertools import chain
import hashlib
import binascii
import sys

In [122]:
#PARAMS
p = 31
Fp = GF(p)
n = 256
k = 52
r = n-k
t = n
z = 3
Fz = GF(z)
lamb = 128
lev_leaf = log(t,2)
Fp_set = Set(Fp)
Fp_star = Fp_set.difference([0])

In [123]:
#FUNZIONI DI UTILITA

def generate_random_seed():
    set_random_seed()
    return initial_seed()

def set_seed(seed):
    set_random_seed(seed)
    return initial_seed()

def hash_and_get_seed(input):
    hash = hashlib.sha256(str(input).encode()).digest()
    seed = int.from_bytes(hash, 'big')
    return seed

def hexhash(input):
    return hashlib.sha256(input.encode()).hexdigest()

def convert_int_to_hash_string(input):
    return input.to_bytes((input.bit_length() + 7) // 8, 'big').hex()

def expand_seed(seed_init):
    seed_str = str(seed_init)
    seeds = hashlib.sha256(seed_str.encode()).digest()
    
    seed_1 = seeds[:16] #i primi 16 byte
    seed_2 = seeds[16:] #gli ultimi 16 byte
    
    seed_1 = int.from_bytes(seed_1, 'big')
    
    seed_2 = int.from_bytes(seed_2, 'big')
    
    return seed_1, seed_2
#t=8
# #w = 5

#0,2,4,5,7

#Creo Generatore g del Campo E
def get_generator(p, z, Fp):
    assert is_prime(p),"p is not prime!"
    assert (p-1)%z == 0, "z does not divide p-1!"
    alpha = Fp.primitive_element()
    #non ricordo perchè exp e g sono scelti secondo questa logica
    exp = (p-1)/z
    g = alpha**exp
    return g

def create_random_vector(domain, n_elems, seed):
    set_seed(seed)
    return random_vector(domain, n_elems)

def create_random_matrix(domain, n_rows, n_columns, seed):
    set_seed(seed)
    M = random_matrix(domain, n_rows, n_columns)
    return M

#Creo vettore errore con elementi in E e lunghezza V 
def create_vEn(g, n, Fz, Fp, seed):
    eta = create_random_vector(Fz, n, seed)
    e = vector(Fp, n)
    for i in range(0, n):#range check
        e[i] = (g**eta[i])
    return e

def create_vector_over_Fp_star(Fp_star, t, seed):
    set_random_seed(seed)
    return [Fp_star.random_element() for i in range(t)]

#def create_ch2_vector(t, seed):

def componentwise_multiply(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("I vettori devono avere la stessa dimensione")
    
    result = vector(GF(p), len(v1))  # Crea un nuovo vettore per i risultati
    for i in range(len(v1)):
        result[i] = v1[i] * v2[i]  # Moltiplica gli elementi corrispondenti
    
    return result

In [124]:
#SEED TREE
class SeedTree:
    def __init__(self, depth, salt, seed_m):
        if(depth>0):
            self.seed_m = seed_m
            self.depth = depth
            self.salt = str(salt)
            self.tree = None
            self.full = False
            self.seed_path = None
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self):
        if self.full:
            raise Exception('Albero gia creato')
        else:
            #BISOGNA AGGIUNGERE CONTROLLI?
            self.tree = []
            self.__make_tree(self.seed_m, 0)
            self.full = True
        
    def get_leaves(self):
        if self.full:
            return self.tree[self.depth]
        else:
            raise Exception('Albero non creato')

    def get_seed_path(self):
        if self.full and self.seed_path is not None:
            return self.seed_path

    def compute_seed_path(self, list_nodes):
        if self.full and len(list_nodes)>0:
            self.seed_path={}
            self.__compute_seed_path(self.depth-1, list_nodes)
            return self.seed_path
        else:
            raise Exception('Albero non creato o foglia non valida')

#DA FARE
    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth<=self.depth):
            list_children = self.__create_new_level(list_nodes, depth)
            self.tree.append(list_children)
            self.__make_tree(list_children, depth+1)

    def __create_new_level(self, parents, level):
        children = []
        if level == 0:
            sx, dx = self.__compute_children_value(parents, '')
            children.append(sx)
            children.append(dx)
        else:
            base = 2**level
            for i in range(0, len(parents)):
                sx, dx = self.__compute_children_value(parents[i], base+i)
                children.append(sx)
                children.append(dx)
        return children
        
    def __compute_children_value(self, seed_parent, index_parent):
        nodes = str(seed_parent) + self.salt + str(index_parent)
        sx, dx = expand_seed(int(nodes))  
        return sx, dx
    
    #provvisorio
    def __compute_seed_path(self, depth, list_index):
        if depth>0:
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))#prendo gli indici dei nodi padri eliminando i duplicati
            list_sub_root = list(filter(lambda x: ((x % 2 == 0 and x + 1 not in list_parent) or (x % 2 != 0 and x - 1 not in list_parent)), list_parent))
            list_new_children = list(filter(lambda x:  x not in list_sub_root, list_parent))
            if len(list_new_children)>0:
                self.seed_path[depth] = dict(map(lambda x: (x, self.tree[depth][x]), list_new_children))
                self.__compute_seed_path(depth-1, list_new_children)
            
        

#DA FARE
    def __verify_proof(self, list_proof, index):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()
 

In [125]:
#MERKLE TREE

class MerkleTree:
    def __init__(self, n_leaves):
        if(n_leaves%2==0):
            self.n_leaves = n_leaves
            self.depth = log(n_leaves, 2)
            self.tree =  None
            self.merkle_proof =  None
            self.full = False
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self, leaves):
        if isinstance(leaves, list) and len(leaves) == self.n_leaves and not(self.full):
            self.tree = []
            self.tree.append(leaves)
            self.__make_tree(leaves, self.depth-1)
            self.full = True
        else:
            raise Exception('Il numero delle foglie deve essere pari a {}'.format(self.n_leaves))
       
    def get_root(self):
        if self.full:
            return self.tree[0][0]
        else:
            raise Exception('Albero non creato')

    def get_merkle_proof(self):
        if self.full and self.merkle_proof is not None:
            return self.merkle_proof

    def compute_proof(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            list_leaves = list(map(lambda x: x+1 if x % 2 == 0 else x-1, list_nodes))
            list_leaves = list(filter(lambda x: x not in list_nodes, list_leaves))
            list_indexes = self.__get_indexes(self.depth, list_leaves, [])
            self.__compute_merkle_proof(list_indexes)
            return self.merkle_proof
        else:
            raise Exception('Albero non creato o foglia non valida')

#DA FARE
    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth>=0):
            list_parent = self.__create_new_level(list_nodes)
            self.tree.insert(0, list_parent)
            self.__make_tree(list_parent, depth-1)

    def __create_new_level(self, child_nodes):
        parents = []
        for i in range(0, len(child_nodes), 2):
            new_node = self.__compute_father_value(child_nodes[i], child_nodes[i+1])
            parents.append(new_node)
        return parents
        
    def __compute_father_value(self, sx, dx):
        return hexhash(sx+dx)
    
    def __get_indexes(self, depth, list_index, list_proof):
        if(depth > 0):
            list_proof.insert(0, list_index)
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))
            return self.__get_indexes(depth-1, list_parent, list_proof)
        else:
            return list_proof


    def __compute_merkle_proof(self, list_merkle_proofs):
        self.merkle_proof = {}    
        for i in range(len(list_merkle_proofs)): 
            current_proof = list_merkle_proofs[i] #controllo livello i esimo
            if i < len(list_merkle_proofs) - 1:
                next_proof = list_merkle_proofs[i+1]
                current_proof = list(filter(lambda x: x*2 not in next_proof and (x*2)+1 not in next_proof, current_proof))
            if not len(current_proof)==0:
                self.merkle_proof[i+1] = dict(map(lambda x: (x, self.tree[i+1][x]), current_proof))

#DA FARE
    def __verify_proof(self, list_proof, index):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()

In [126]:
#GENERAZIONE CHIAVI
def key_gen():
    seed_sk = generate_random_seed() 
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk) 
    g = get_generator(p, z, Fp)
    e = create_vEn(g, n, Fz, Fp, seed_e)
    s = e[0:r] + e[r:n]*V
    return seed_sk, seed_pk, s


In [155]:

class Signer:
    def __init__(self, seed_sk, seed_pk, s):
        self.seed_sk = seed_sk
        self.seed_pk = seed_pk
        self.seed_e = None
        self.salt = None
        self.V = None
        self.eta = None
        self.s = s
        self.seed_tree = None
        self.merkle_tree = None
        self.c0 = None
        self.c1 = None
        self.ch1 = None
        self.ch2 = None
        self.h = None



    def key_material_expansion(self):
        self.seed_e , self.seed_pk = expand_seed(self.seed_sk)
        self.V = create_random_matrix(Fp, k, r, self.seed_pk)
        self.eta = create_random_vector(Fz, n, self.seed_e)

    def compute_commitment(self):
        self.seed_m = generate_random_seed() 
        seed_salt = generate_random_seed()
        self.salt = hash_and_get_seed(seed_salt)
        self.seed_tree = SeedTree(lev_leaf, self.salt, self.seed_m)
        self.seed_tree.make_tree()
        self.seeds = self.seed_tree.get_leaves()
        cmt0 = [] 
        cmt1 = []
        g = get_generator(p, z, Fp)
        for i in range(0, t):
            seed_ui, seed_ei = expand_seed(self.seeds[i])
            eta_i = create_random_vector(Fz, n, seed_ei)
            sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
            v = vector(Fp,n) #v∈ Fp^n
            for j in range(0, n-1):
                v[j] = (g**sigma_i[j])  
            ui = create_random_vector(Fp, n, seed_ui)
            u = componentwise_multiply(v, ui)
            st = u[0:r] + u[r:n]*self.V
            commitment0 = hexhash( str(st) + str(sigma_i) + str(self.salt) + str(i) )
            commitment1 = hexhash( str(self.seeds[i]) + str(self.salt) + str(i) )
            cmt0.append(commitment0) 
            cmt1.append(commitment1)
        self.merkle_tree = MerkleTree(t)
        self.merkle_tree.make_tree(cmt0)
        self.c0 = self.merkle_tree.get_root()
        input_c1 = "".join(map(str, cmt1))
        self.c1 = hexhash(input_c1)

    def generate_first_challenge(self, msg):
        hash_salt = convert_int_to_hash_string(self.salt)
        input_ch1 = self.c0+self.c1+msg+hash_salt
        seed_ch1 = hash_and_get_seed(input_ch1)
        self.ch1 = create_vector_over_Fp_star(Fp_star, t, seed_ch1)

    def generate_first_response(self):
        list_hash_yi=[]
        for seed_i, ch_i_1 in zip(self.seeds, self.ch1):
            seed_ui, seed_ei = expand_seed(seed_i)
            g = get_generator(p, z, Fp)
            ei = create_vEn(g, n, Fz, Fp, seed_ei)
            ui = create_random_vector(Fp, n, seed_ui)
            yi = ui+(ch_i_1)*ei
            hi = hexhash(str(yi))
            list_hash_yi.append(hi)
        self.h = hexhash("".join(list_hash_yi))

    def generate_second_challenge(self, msg):#, seed_tree):
        hash_salt = convert_int_to_hash_string(self.salt)
        input_ch2 = self.c0+self.c1+"".join(map(str, self.ch1))+self.h+msg+hash_salt
        seed_ch2 = hash_and_get_seed(input_ch2)
        self.ch2 = create_random_vector(GF(2), t, seed_ch2)
        #self.ch2 = create_ch2_vector(t, seed_ch2)

    def generate_second_response(self):
        index_1, index_0 = [], []
        rsp0 = []
        rsp1 = []
        print(len(self.ch2))
        print(len(self.c1))
        g = get_generator(p, z, Fp)
        for i, x in enumerate(self.ch2):
            if x == 0:
                index_0.append(i)
                seed_ui, seed_ei = expand_seed(self.seeds[i])
                eta_i = create_random_vector(Fz, n, seed_ei)
                sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
                ei = create_vEn(g, n, Fz, Fp, seed_ei)
                ui = create_random_vector(Fp, n, seed_ui)
                yi = ui+(self.ch1[i])*ei
                rsp0.append([yi, sigma_i])
            else:
                index_1.append(i)
                rsp1.append(hexhash( str(self.seeds[i]) + str(self.salt) + str(i) ))
        [index_1.append(i) if x == 1 else index_0.append(i) for i, x in enumerate(self.ch2)] 
        seed_path = self.seed_tree.compute_seed_path(index_1)
        merkle_proofs = self.merkle_tree.compute_proof(index_0)
        return merkle_proofs, seed_path, rsp0, rsp1

#    def pseudo():
        #for i ← 0 to t − 1 do
            #if (b[i] = 0) then
                #cmt0[i] is recomputed by the verifier, cmt1[i] must be sent
                #rsp0[j] ← (yi, δi)  rsp0[j] ← (yi, σi)
                #rsp1[j] ← cmt1[i]
                #j ← j + 1
            #end
        #end
    def sign_msg(self, msg):
        self.key_material_expansion()
        self.compute_commitment()
        self.generate_first_challenge(msg)
        self.generate_first_response()
        self.generate_second_challenge(msg)
        self.generate_second_response()


In [156]:
seed_sk, seed_pk, s = key_gen()
signer = Signer(seed_sk, seed_pk, s)
signer.sign_msg("buonasera")

256
64
RSP0
[[(8, 25, 13, 16, 22, 16, 21, 7, 29, 18, 7, 16, 10, 14, 12, 20, 14, 5, 29, 27, 13, 12, 27, 20, 17, 13, 16, 22, 21, 21, 17, 14, 19, 29, 3, 22, 15, 29, 15, 15, 29, 17, 18, 27, 22, 21, 5, 11, 21, 1, 0, 24, 22, 21, 4, 11, 22, 29, 30, 26, 30, 10, 2, 11, 22, 25, 7, 14, 28, 7, 28, 22, 19, 30, 15, 23, 7, 8, 25, 6, 4, 13, 23, 3, 24, 17, 17, 22, 13, 29, 29, 16, 9, 22, 18, 0, 10, 27, 1, 18, 6, 8, 19, 19, 30, 13, 27, 21, 21, 11, 9, 15, 6, 21, 1, 15, 23, 26, 2, 11, 27, 19, 4, 27, 22, 1, 30, 0, 19, 12, 20, 8, 18, 0, 9, 30, 0, 16, 3, 30, 16, 0, 7, 16, 8, 18, 28, 5, 16, 3, 16, 11, 11, 12, 26, 1, 17, 20, 1, 4, 21, 8, 30, 4, 27, 15, 30, 30, 0, 4, 1, 28, 12, 20, 8, 12, 4, 11, 4, 22, 23, 2, 20, 0, 26, 9, 9, 12, 7, 23, 14, 3, 6, 11, 6, 23, 17, 30, 21, 10, 1, 28, 30, 16, 17, 10, 10, 19, 23, 14, 26, 7, 11, 1, 24, 24, 21, 9, 12, 14, 18, 8, 1, 17, 17, 6, 4, 23, 21, 19, 13, 7, 30, 16, 15, 21, 10, 13, 5, 5, 18, 29, 5, 11, 14, 16, 14, 22, 29, 18, 13, 8, 12, 0, 16, 15), (0, 2, 0, 0, 2, 1, 2, 2, 1, 0, 2

In [129]:
#Commitment 0
print(signer.c0)  


bb90be17e951446d9a212149650951b2aa1d66db471ee52333c706d083ad00f9


In [130]:
#Commitment 1
print(signer.c1) 



aa12b9ca9c2e37621609233d3b2fcd87ead4792aadc50e1f47d5fc46deb992f0


In [131]:
#Challenge 1
print(signer.ch1) 


[21, 1, 23, 6, 15, 9, 19, 9, 4, 22, 29, 18, 15, 30, 1, 14, 28, 15, 17, 22, 27, 6, 11, 3, 28, 17, 2, 6, 22, 23, 2, 7, 29, 14, 16, 4, 6, 2, 16, 5, 27, 16, 16, 7, 25, 10, 10, 10, 17, 11, 3, 29, 14, 14, 12, 2, 25, 5, 25, 21, 11, 5, 25, 10, 19, 25, 2, 15, 8, 6, 10, 20, 7, 1, 17, 26, 29, 7, 16, 5, 28, 23, 12, 9, 20, 9, 7, 9, 28, 21, 1, 29, 15, 13, 28, 22, 5, 7, 21, 15, 21, 27, 2, 12, 9, 27, 26, 13, 6, 28, 29, 20, 10, 8, 6, 3, 6, 25, 10, 26, 19, 15, 17, 29, 15, 10, 12, 19, 6, 13, 17, 10, 23, 30, 19, 2, 11, 15, 15, 15, 9, 16, 20, 28, 9, 6, 7, 27, 16, 29, 27, 21, 4, 26, 20, 28, 2, 8, 16, 14, 15, 14, 3, 26, 26, 8, 6, 13, 2, 22, 14, 15, 16, 22, 11, 15, 6, 19, 21, 22, 15, 2, 11, 15, 29, 6, 1, 6, 23, 29, 28, 5, 15, 6, 30, 23, 15, 6, 4, 16, 1, 23, 24, 28, 20, 21, 11, 4, 14, 9, 24, 16, 7, 22, 30, 6, 22, 1, 15, 26, 24, 28, 4, 21, 22, 16, 21, 26, 27, 3, 28, 11, 25, 27, 10, 25, 10, 27, 4, 3, 3, 13, 13, 16, 13, 6, 23, 4, 28, 15, 8, 11, 30, 15, 16, 17]


In [132]:
#Hash 1
print(signer.h)


885652acdfe423dd3afdbc24ebe3e1aabee3dbd63821fc4647b2872fdc2e50b1


In [133]:
#Challenge 2
print(signer.ch2) 


(1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0)


In [134]:
#Seed Path che contiene l'insieme dei nodi dei sottoalberi per ricostruire i seed (NON SO SE GIUSTO)
#è stata fatta come un dizionario chiave valore
seed_path = signer.seed_tree.get_seed_path()
size_in_bytes = sys.getsizeof(seed_path)
size_in_bits = size_in_bytes * 8

In [135]:
#Merkle proof che contiene l'insieme dei nodi dei sottoalberi per ricostruire le foglie (DA GLI STESSI RISULTATI DEL TUO) 
#è stato fatta come un dizionario chiave valore
merkle_proof = (signer.merkle_tree.get_merkle_proof())
size_in_bytes = sys.getsizeof(merkle_proof)
size_in_bits = size_in_bytes * 8